# Gaussian Discriminant Analysis
    < Andrew Carr >
    Math 404
    February 7, 2018

In [1]:
import numpy as np
import time

from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

from sklearn.datasets import load_breast_cancer

Helper Function

In [3]:
def get_time(func, label, train=False, score=False):
    """simplify timing by adding a function to do it for me."""
    print("\n{}:\n".format(label))
    
    # determine type of action
    if train:
        type_of_action = "train"
    else:
        type_of_action = "test"
        
    # run the function
    start = time.time()
    if train:
        func(train_x,train_y)
    else:
        func(test_x,test_y)
    print("\t{} time {}".format(type_of_action, time.time() - start))
    if score:
        print("\taccuracy {}".format(func(test_x, test_y)))

# Problem 1
Code up the Gaussian Discriminant Analysis algorithm.  Your code should have a .fit method that accepts a dataset X,y where y only takes on a finite number of values (classes), the .fit method should train the model (learn the parameters $\pi_c$, $\mu_c$, and  $\Sigma_c$ for each class c, using the standard Gaussian MLE for each $\mu_c$, and  $\Sigma_c$ and using the estimate $\pi_c$ = (#y=c)/N.  Your code should also have a `.predict_proba` method that accepts a data set X' and returns $p(y=c | x)$ for each x in X',  and it should have a `.predict` method that accepts data X' and returns the class prediction $\hat{y}$ for each x in X' 

In [2]:
class GDA(object):
    """Gaussian Discriminant Analysis Classifier Object"""
    def __init__(self):
        pass
    
    def fit(self, x,y):
        """Fit a GDA model given features and labels, returns NONE"""
        self.train_x = x
        self.train_y = y
        self.num_samples,self.num_features = self.train_x.shape
        self.classes = list(set(y))
        self.stats = {}
        # for both classes
        for c in self.classes:
            features = self.train_x[self.train_y == c]
            
            # calculate the probability of the class
            pi_c = len(features)/len(self.train_x)
            
            # gather statistics
            mean = np.mean(features, axis=0)
            cov = np.cov(features,rowvar=False)
            
            # create
            self.stats[str(c)] = [stats.multivariate_normal(mean=mean, cov=cov, allow_singular=True), pi_c]
         
    def predict_proba(self,x):
        """Calculate relative probability (doesn't sum to 1) of x over classes, returns matrix of probabilities"""
        self.test_x = x
        self.probs = []
        
        # for all of our data
        for instance in self.test_x:
            
            # get all class probabilities
            internal_probs = []
            for c in self.classes:
                
                # gather info from fit and save
                norm, pi_c = self.stats[str(c)]
                prob = norm.pdf(instance)*pi_c
                internal_probs.append(prob)
                
            self.probs.append(internal_probs/np.sum(internal_probs)) # row normalize to sum to 1
        return self.probs
                
        
    def predict(self, x):
        """Calculate class prediction of x over classes, returns list of predictions"""
        self.test_x = x
        self.prediction = []
        # get probabilities
        self.predict_proba(x)
        
        # find the class by taking the argmax, works since these are relative probabilities
        self.prediction = np.argmax(self.probs, axis=1)
        return self.prediction

        
    def accuracy(self, y_true,y_pred):
        """Find accuracy on a test set returns float, this assumes classes are ordered and not randomly ordered"""
        return accuracy_score(y_true, y_pred)
    
    def score(self, test_x, test_y):
        """predicts and scores returns float"""
        self.predict(test_x)
        return self.accuracy(test_y, self.prediction)
        

# Problem 2
Apply your GDA code to the cancer dataset with an appropriate train-test split and compare the results (train and test speed and test accuracy) to logistic regression and Naive Bayes.  Is one of these much better than the others?  Explain. 

In [4]:
data = load_breast_cancer()

gda_clf = GDA()
nb_clf = GaussianNB()
log_clf = LogisticRegression()

train_x, test_x, train_y, test_y = train_test_split(data.data,data.target, train_size=0.7)

get_time(gda_clf.fit, label="GDA", train=True)
get_time(nb_clf.fit, label="NB", train=True)
get_time(log_clf.fit, label="Logistic Regression", train=True)

get_time(gda_clf.score, label="GDA", score=True)
get_time(nb_clf.score, label="NB", score=True)
get_time(log_clf.score, label="Logistic Regression", score=True)


GDA:

	train time 0.001970052719116211

NB:

	train time 0.0013041496276855469

Logistic Regression:

	train time 0.004260063171386719

GDA:

	test time 0.013648748397827148
	accuracy 0.9415204678362573

NB:

	test time 0.0005729198455810547
	accuracy 0.9064327485380117

Logistic Regression:

	test time 0.0003521442413330078
	accuracy 0.935672514619883


We see that they take a similar amount of time to train, with naive bayes being slightly faster. However, both SKlearn packages test much faster, while accuracy goes back and forth depending on the split. In this case my GDA had higher accuracy, but they are comparable. 

# Problem 3
Compare your train and test speed and your test accuracy to the `discriminant_analysis.QuadraticDiscriminantAnalysis` method in scikit learn. 

In [5]:
gda_clf = GDA()
sk_gda_clf = QuadraticDiscriminantAnalysis()

get_time(gda_clf.fit, label="GDA", train=True)
get_time(sk_gda_clf.fit, label="SKlearn QDA", train=True)

get_time(gda_clf.score, label="GDA", score=True)
get_time(sk_gda_clf.score, label="SKlearn QDA", score=True)


GDA:

	train time 0.0015871524810791016

SKlearn QDA:

	train time 0.0017061233520507812

GDA:

	test time 0.05335497856140137
	accuracy 0.9415204678362573

SKlearn QDA:

	test time 0.0007081031799316406
	accuracy 0.9532163742690059


In this case we see, as expected that train time is relatively similar since my code is nice and vectorized for speed. However, test time is very different. There's is much faster and has higher accuracy. This is super interesting to see. 